# **SQL Murder Mystery**

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

![](schema.png)

In [2]:
%load_ext sql
%sql sqlite:///sql-murder-mystery.db

In [3]:
%%sql

SELECT name 
FROM sqlite_master
WHERE TYPE = 'view'

 * sqlite:///sql-murder-mystery.db
Done.


name
witnesses
murderer


In [4]:
%%sql

DROP VIEW murderer

 * sqlite:///sql-murder-mystery.db
Done.


[]

## Part 1

Step one is to find the crime scene report corresponding to our murder. 

In [5]:
%%sql

SELECT description FROM crime_scene_report
WHERE 
    city = "SQL City" 
    AND date = 20180115
    AND type = "murder"

 * sqlite:///sql-murder-mystery.db
Done.


description
"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


### Witnesses

In [6]:
%%sql

CREATE VIEW witnesses AS
SELECT * FROM person 
WHERE 
    (
        address_number = (
            SELECT MAX(address_number) FROM person 
            WHERE address_street_name = "Northwestern Dr"
        )
    )
    OR (
        address_street_name = "Franklin Ave" 
        AND name LIKE "Annabel%"
    );

SELECT * FROM witnesses

 * sqlite:///sql-murder-mystery.db
(sqlite3.OperationalError) view witnesses already exists
[SQL: CREATE VIEW witnesses AS
SELECT * FROM person 
WHERE 
    (
        address_number = (
            SELECT MAX(address_number) FROM person 
            WHERE address_street_name = "Northwestern Dr"
        )
    )
    OR (
        address_street_name = "Franklin Ave" 
        AND name LIKE "Annabel%"
    );]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Interviews

In [7]:
%%sql 

SELECT * FROM interview
WHERE person_id IN (SELECT id FROM witnesses)

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


### Gym & License Plate

In [8]:
%%sql 

CREATE VIEW murderer AS
SELECT * FROM person 
WHERE 
    id IN (
        SELECT person_id FROM get_fit_now_member
        WHERE  (id LIKE "48Z%" and membership_status = "gold")
    ) 
    AND
    license_id IN (
        SELECT id FROM drivers_license
        WHERE 
            plate_number LIKE "%H42W%"
            AND gender = "male"
    );

SELECT * FROM murderer

 * sqlite:///sql-murder-mystery.db
Done.
Done.


id,name,license_id,address_number,address_street_name,ssn
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [9]:
%%sql 

INSERT INTO solution VALUES (1, (SELECT name FROM murderer));        
SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
Done.


value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


In [10]:
%%sql 

SELECT * FROM interview
WHERE person_id = (SELECT id FROM murderer)

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [11]:
%%sql 

SELECT * FROM drivers_license
WHERE 
    height BETWEEN 65 AND 67
    AND car_make = "Tesla"
    AND hair_color = "red"

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
202298,68,66,green,red,female,500123,Tesla,Model S
291182,65,66,blue,red,female,08CM64,Tesla,Model S
918773,48,65,black,red,female,917UU3,Tesla,Model S


In [28]:
%%sql 

CREATE VIEW villain AS
SELECT * FROM person 
WHERE 
    license_id IN (
        SELECT id FROM drivers_license
        WHERE 
            height BETWEEN 65 AND 67
            AND car_make = "Tesla"
            AND hair_color = "red"
    )
    AND
    id IN (
        SELECT person_id FROM (
            SELECT person_id, COUNT(*) AS count 
            FROM facebook_event_checkin
            WHERE date LIKE "201712%" 
            GROUP BY person_id
            HAVING count = 3
        )
    );

SELECT * FROM villain



 * sqlite:///sql-murder-mystery.db
Done.
Done.


id,name,license_id,address_number,address_street_name,ssn
99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [30]:
%%sql 

INSERT INTO solution VALUES (1, (SELECT name FROM villain));        
SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
Done.


value
"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"


In [25]:
%%sql 

SELECT person_id FROM (
    SELECT person_id, COUNT(*) AS count 
    FROM facebook_event_checkin
    WHERE date LIKE "201712%" 
    GROUP BY person_id
    HAVING count = 3
)


 * sqlite:///sql-murder-mystery.db
Done.


person_id
23035
24556
66276
99716


In [13]:
%%sql

SELECT * FROM myview

 * sqlite:///sql-murder-mystery.db
(sqlite3.OperationalError) no such table: myview
[SQL: SELECT * FROM myview]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [14]:
%%sql 

SELECT * FROM get_fit_now_member
WHERE 
    (id LIKE "48Z%" and membership_status = "gold")
    AND 

 * sqlite:///sql-murder-mystery.db
(sqlite3.OperationalError) incomplete input
[SQL: SELECT * FROM get_fit_now_member
WHERE 
    (id LIKE "48Z%" and membership_status = "gold")
    AND]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [15]:
%%sql

SELECT * FROM get_fit_now_check_in 
WHERE membership_id = (
    SELECT id FROM get_fit_now_member
    WHERE person_id = 16371
)

 * sqlite:///sql-murder-mystery.db
Done.


membership_id,check_in_date,check_in_time,check_out_time
90081,20180109,1600,1700


In [16]:
%%sql 

SELECT * FROM get_fit_now_check_in
WHERE 
    check_out_time > (
        SELECT check_in_time FROM get_fit_now_check_in 
        WHERE membership_id = (
            SELECT id FROM get_fit_now_member
            WHERE person_id = 16371
        )
    )

    AND check_in_time < (
        SELECT check_out_time FROM get_fit_now_check_in 
        WHERE membership_id = (
            SELECT id FROM get_fit_now_member
            WHERE person_id = 16371
        )
    )



 * sqlite:///sql-murder-mystery.db
Done.


membership_id,check_in_date,check_in_time,check_out_time
48Z7A,20180109,1600,1730
48Z55,20180109,1530,1700
90081,20180109,1600,1700
